# Cadastro Geral

Atualmente há duas lista de cadastro geral no Ponto do Bem. O objetivo desse programa é conciliar os dados das duas em uma planilha só. Ademais, userei VBA para corresponder dados novos em futuras atualizaçôes.



In [1]:
import pandas as pd

In [26]:
geral_oficial = pd.read_csv("/content/CADASTRO_GERAL.csv", parse_dates=True)
geral_nao_oficial = pd.read_csv("/content/Cadastro_Geral_Nao_oficial.csv", parse_dates=True)

## Visualizações

In [ ]:
geral_nao_oficial.head()

,Nº,Nome,Liberado?,Remédios / Roupas,Bairro,Telefone,Crianças,Adultos,Idosos,Trabalha,Tipo de Auxílio,Qtde de pessoas,(R$),Data de Atualização,Endereço,CPF,DATA DA PLANILHA,Observações
0,1.0,Abdaline Jean Baptiste,NaN,NaN,Tanque,98064-1512 - 97201-0948,3,2,2,não,NaN,7,NaN,6/19/2020,"Rua Almirante Melquíades de Souza, 322 Tanque ...",065.001.667-01,7/3/2020,NaN
1,2.0,Abigail Pereira Cabral,NaN,NaN,Praça Seca,998039-7091 (tel errado),1,3,NaN,não,NaN,4,NaN,6/5/2020,NaN,714.666.306-34,7/3/2020,NaN
2,3.0,Adélia Verônica Pinto da Silva,OK,Sim,Vila Isabel,99312-1634,1,2,0,não,"bolsa famila 130,00",3,NaN,4/20/2021,Rua viana drumond 38 casa 2 Vila Isabel,125.458..797-70,7/11/2020,NaN
3,4.0,Adelino da Silva Gomes,OK RECEBEU,NaN,Vila Isabel,98891-2516,3,1,NaN,NaN,NaN,4,NaN,NaN,"Rua Armando de Albuquerque, 33",011.833.607-03,6/27/2020,NaN
4,5.0,Aderaldo Araújo Simplício,NaN,NaN,Tijuca,99947-3523,NaN,NaN,NaN,NaN,NaN,0,NaN,7/25/2020,NaN,606.569.647-15,8/8/2020,NaN


In [ ]:
geral_oficial.head()

,Nº+,Nome,EV/AB/PRAÇA,Pedidos de Roupa/Remédio,Bairro,Telefone,Crianças,Adultos,Idosos,Trabalha,Recebe Auxílio?,Qtde de pessoas,Renda mensal,Data de entrada,Endereço,CPF,CESTA 1,CESTA 2,CESTA 3,Última atualização,Observações
0,1,Abdaline Jean Baptiste,Praça,Não,Tanque,98064-1512,4,1,2,Não,Não,7,"1.100,00",???,Rua Almirante de Souza - Casa 322,065.001.667-01,NaN,NaN,NaN,22/03/2021,NaN
1,2,Abigail Pereira Cabral,NaN,NaN,Praça Seca,998039-7091 (tel errado),1,3,NaN,não,NaN,4,NaN,05/06/2020,NaN,714.666.306-34,NaN,NaN,NaN,03/07/2020,NaN
2,3,Adélia Verônica Pinto da Silva,AB,Sim,Vila Isabel,99312-1634,1,1,1,Não,Não,3,"0,00",11/08/2020,"Rua Senador Nabuco, 411",125.458.797-70,NaN,NaN,NaN,11/07/2020,NaN
3,4,Adelino da Silva Gomes,Praça,Sim,Vila Isabel,(21) 98891-2516,2,2,0,Não,Não,4,"0,00",29/05/2020,"Rua Armando de Albuquerque, 33",011.833.607-03,NaN,NaN,NaN,29/05/2020,NaN
4,5,Aderaldo Araújo Simplício,NaN,NaN,Tijuca,99947-3523,NaN,NaN,NaN,NaN,NaN,0,NaN,25/07/2020,NaN,606.569.647-15,NaN,NaN,NaN,08/08/2020,NaN


Vemos que a lista oficial tem mais dados do que a não oficial.

In [ ]:
geral_nao_oficial.shape, geral_oficial.shape

((1368, 18), (1541, 21))

In [ ]:
geral_oficial.dtypes

Nº+                          object
Nome                         object
EV/AB/PRAÇA                  object
Pedidos de Roupa/Remédio     object
Bairro                       object
Telefone                     object
Crianças                     object
Adultos                      object
Idosos                       object
Trabalha                     object
Recebe Auxílio?              object
Qtde de pessoas              object
Renda mensal                 object
Data de entrada              object
Endereço                     object
CPF                          object
CESTA 1                      object
CESTA 2                      object
CESTA 3                     float64
Última atualização           object
Observações                  object
dtype: object

A lista oficial foi toda reconhecida como string, exceto a coluna da Cesta 3. 

In [ ]:
geral_nao_oficial.dtypes

Nº                     float64
Nome                    object
Liberado?               object
Remédios / Roupas       object
Bairro                  object
Telefone                object
Crianças                object
Adultos                 object
Idosos                  object
Trabalha                object
Tipo de Auxílio         object
Qtde de pessoas         object
(R$)                    object
Data de Atualização     object
Endereço                object
CPF                     object
DATA DA PLANILHA        object
Observações             object
dtype: object

## Modificando e validando dados

### Excluindo e Renomeando colunas

In [27]:
geral_nao_oficial = geral_nao_oficial.drop("Liberado?", axis = 1) # Exluiu a coluna Liberado?
geral_nao_oficial = geral_nao_oficial.drop("Nº", axis = 1) # Exclui a coluna Nº

Para melhor usar o join/merge/combine faremos as colunas nas duas planilhas terem o mesmo nome quando necessárias.

In [28]:
nomes = {"Remédios / Roupas": "Pedidos de Roupa/Remédio", "Tipo de Auxílio": "Recebe Auxílio?",
         "(R$)": "Renda mensal", "DATA DA PLANILHA": "Data de entrada", "Data de Atualização": "Última atualização"}

In [29]:
geral_nao_oficial.rename(nomes, axis = 1, inplace=True)

In [30]:
geral_nao_oficial.head()

,Nome,Pedidos de Roupa/Remédio,Bairro,Telefone,Crianças,Adultos,Idosos,Trabalha,Recebe Auxílio?,Qtde de pessoas,Renda mensal,Última atualização,Endereço,CPF,Data de entrada,Observações
0,Abdaline Jean Baptiste,NaN,Tanque,98064-1512 - 97201-0948,3,2,2,não,NaN,7,NaN,6/19/2020,"Rua Almirante Melquíades de Souza, 322 Tanque ...",065.001.667-01,7/3/2020,NaN
1,Abigail Pereira Cabral,NaN,Praça Seca,998039-7091 (tel errado),1,3,NaN,não,NaN,4,NaN,6/5/2020,NaN,714.666.306-34,7/3/2020,NaN
2,Adélia Verônica Pinto da Silva,Sim,Vila Isabel,99312-1634,1,2,0,não,"bolsa famila 130,00",3,NaN,4/20/2021,Rua viana drumond 38 casa 2 Vila Isabel,125.458..797-70,7/11/2020,NaN
3,Adelino da Silva Gomes,NaN,Vila Isabel,98891-2516,3,1,NaN,NaN,NaN,4,NaN,NaN,"Rua Armando de Albuquerque, 33",011.833.607-03,6/27/2020,NaN
4,Aderaldo Araújo Simplício,NaN,Tijuca,99947-3523,NaN,NaN,NaN,NaN,NaN,0,NaN,7/25/2020,NaN,606.569.647-15,8/8/2020,NaN


### Estruturando a Coluna "CPF"

In [96]:
# Essa função enviar cada valor para ser checado
def separacao(data_serie):
  num = 0

  for i in range(len(data_serie)):
    try:
      if math.isnan(data_serie[i]):
        # Se o valor for NaN ele pula
        continue
        
    except TypeError:
        pass

    if type(data_serie[i]) is list:
      # Se for uma lista, junta os valores ao converte-los para string
      data_serie[i] = [str(x) for x in data_serie[i]]
      data_serie[i] = "".join(data_serie[i])

    if data_serie[i].startswith("ID:") or data_serie[i].startswith("RG"): # Se começa com "ID:" é ignorado
      continue

    valores = cpf(data_serie[i])
    num += 1

    if num % 10 == 0:
      # Printa a cada 10 CPF
      print(data_serie[i], valores[0], "index =", i)

    if valores[1] == False:
      data_serie[i] = valores[0]


In [90]:
import math
# Essa função checará se o CPF está no formato XXX.XXX.XXX-XX
def cpf(valor):
  
  cpf = []
  valor = list(valor.strip()) # Transforma em uma lista
      
  for i in range(len(valor)):
    # Começa a verificar do ultimo ao primeiro
        
    if valor[i].isdigit():
      # Se o valor for digito

      cpf.append(str(valor[i]))
    
  while len(cpf) < 11:
    # se a lista for menor que 11 será adicionado um 0 até completar 11 digito

    cpf.insert(0, "0")
  
  # Insere os caracteres nas posições específicas
  cpf.insert(3, ".") 
  cpf.insert(7, ".")
  cpf.insert(11, "-")

  return ["".join(cpf), False]


    


In [97]:
# Passando a serie da Coluna CPF
coluna_cpf = geral_nao_oficial["CPF"]
coluna_cpf = separacao(coluna_cpf)

127.383.047-45 127.383.047-45 index = 13
002.106.317-67 002.106.317-67 index = 25
014.187.861-44 014.187.861-44 index = 46
171.637.447-01 171.637.447-01 index = 67
035.672.627-42 035.672.627-42 index = 101
025.869.567-61 025.869.567-61 index = 126
917.828.487-20 917.828.487-20 index = 145
884.896.713-00 884.896.713-00 index = 165
149.072.277-79 149.072.277-79 index = 190
098.072.117-27 098.072.117-27 index = 219
144.283.937-94 144.283.937-94 index = 240
821.274.767-00 821.274.767-00 index = 261
081.657.907-57 081.657.907-57 index = 288
047.671.187-86 047.671.187-86 index = 312
112.168.227-86 112.168.227-86 index = 325
722.627.887-15 722.627.887-15 index = 345
098.865.774-01 098.865.774-01 index = 362
098.991.417-80 098.991.417-80 index = 396
158.517.007-00 158.517.007-00 index = 414
114.107.417-61 114.107.417-61 index = 432
074.763.917-57 074.763.917-57 index = 452
133.796.307-05 133.796.307-05 index = 468
078.310.637-81 078.310.637-81 index = 490
117.478.047-97 117.478.047-97 index = 

In [98]:
coluna_cpf_oficial = separacao(geral_oficial["CPF"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


150.421.007-76 150.421.007-76 index = 9
095.088.617-37 095.088.617-37 index = 19
408.319.067-15 408.319.067-15 index = 31
119.015.767-52 119.015.767-52 index = 46
175.569.847-02 175.569.847-02 index = 58
142.276.967-75 142.276.967-75 index = 79
045.292.207-02 045.292.207-02 index = 102
063.654.317-08 063.654.317-08 index = 120
000.000.000-00 000.000.000-00 index = 144
119.023.867-52 119.023.867-52 index = 159
054.909.144-08 054.909.144-08 index = 178
264.787.558-86 264.787.558-86 index = 203
182.200.997-98 182.200.997-98 index = 230
112.771.037-04 112.771.037-04 index = 245
052.780.027-94 052.780.027-94 index = 264
821.274.767-00 821.274.767-00 index = 279
102.878.267-59 102.878.267-59 index = 302
085.764.904-39 085.764.904-39 index = 318
076.149.067-13 076.149.067-13 index = 336
115.277.217-10 115.277.217-10 index = 350
108.139.547-60 108.139.547-60 index = 369
037.649.427-11 037.649.427-11 index = 381
025.896.447-28 025.896.447-28 index = 402
089.455.647-99 089.455.647-99 index = 423

## Juntando as duas planilhas

In [20]:
Colunas = ["CPF", "Nome", "Telefone", "Data de entrada"]

In [31]:
mergido = pd.merge(geral_nao_oficial, geral_oficial, on = Colunas, how="outer")

In [32]:
geral_nao_oficial.shape, geral_oficial.shape, mergido.shape

((1368, 16), (1541, 21), (2848, 33))

In [34]:
mergido.tail()

,Nome,Pedidos de Roupa/Remédio_x,Bairro_x,Telefone,Crianças_x,Adultos_x,Idosos_x,Trabalha_x,Recebe Auxílio?_x,Qtde de pessoas_x,Renda mensal_x,Última atualização_x,Endereço_x,CPF,Data de entrada,Observações_x,Nº+,EV/AB/PRAÇA,Pedidos de Roupa/Remédio_y,Bairro_y,Crianças_y,Adultos_y,Idosos_y,Trabalha_y,Recebe Auxílio?_y,Qtde de pessoas_y,Renda mensal_y,Endereço_y,CESTA 1,CESTA 2,CESTA 3,Última atualização_y,Observações_y
2843,ZILDA SOARES,NaN,NaN,99551-2776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1502,NaN,NaN,Praça Seca,2,2,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,08/05/2020,NaN
2844,ZILMA DA SILVA MAIA,NaN,NaN,99428-8934/97397-3521 (vizinha),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1503,NaN,NaN,Praça Seca,NaN,2,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,08/05/2020,NaN
2845,ZULEICA SILVA DO JANUÁRIO,NaN,NaN,99193-5560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,22/05/2020,NaN
2846,LORREYNE FARIA DA SILVA,NaN,NaN,97448-5093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163.294.047-71,19/05/2021,NaN,1505,AB,NaN,Tijuca,2,2,0,não,não,4,"375,00","Rua Dona Sinha, 1 - casa 8",NaN,NaN,NaN,20/05/2021,NaN
2847,RELAYNE DA CONCEIÇÃO DUARTE,NaN,NaN,96657-5110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,?,19/05/2021,NaN,#REF!,AB,NaN,Tijuca,5,1,1,não,Bolsa familia,7,"405,00","Rua Joaquim Pizarro, 18",NaN,NaN,NaN,20/05/2021,NaN


* Sem regularizar o CPF são 1678 pessoas no total

In [43]:
combinado = geral_oficial.combine_first(geral_nao_oficial)

In [44]:
combinado.shape

(1541, 21)

In [45]:
combinado.head()

,Adultos,Bairro,CESTA 1,CESTA 2,CESTA 3,CPF,Crianças,Data de entrada,EV/AB/PRAÇA,Endereço,Idosos,Nome,Nº+,Observações,Pedidos de Roupa/Remédio,Qtde de pessoas,Recebe Auxílio?,Renda mensal,Telefone,Trabalha,Última atualização
0,1,Tanque,NaN,NaN,NaN,065.001.667-01,4,???,Praça,Rua Almirante de Souza - Casa 322,2,Abdaline Jean Baptiste,1,NaN,Não,7,Não,"1.100,00",98064-1512,Não,22/03/2021
1,3,Praça Seca,NaN,NaN,NaN,714.666.306-34,1,05/06/2020,NaN,NaN,NaN,Abigail Pereira Cabral,2,NaN,NaN,4,NaN,NaN,998039-7091 (tel errado),não,03/07/2020
2,1,Vila Isabel,NaN,NaN,NaN,125.458.797-70,1,11/08/2020,AB,"Rua Senador Nabuco, 411",1,Adélia Verônica Pinto da Silva,3,NaN,Sim,3,Não,"0,00",99312-1634,Não,11/07/2020
3,2,Vila Isabel,NaN,NaN,NaN,011.833.607-03,2,29/05/2020,Praça,"Rua Armando de Albuquerque, 33",0,Adelino da Silva Gomes,4,NaN,Sim,4,Não,"0,00",(21) 98891-2516,Não,29/05/2020
4,NaN,Tijuca,NaN,NaN,NaN,606.569.647-15,NaN,25/07/2020,NaN,NaN,NaN,Aderaldo Araújo Simplício,5,NaN,NaN,0,NaN,NaN,99947-3523,NaN,08/08/2020


Vemos que usando o ***combine()*** é a melhor opção, já que ele combina os valores faltantes quando as duas colunas tem o mesmo nome. No total temos o mesmo número de colunas e linhas que a planilha Geral_Oficial. Apesar disso, as colunas ficaram embaralhadas, nada que um ordernação não resolva. 

### Reordenando as colunas